<a href="https://colab.research.google.com/github/So-AI-love/chatgpt-prompts-for-academic-writing/blob/main/Auto_Making/Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free

by [Max Woolf](http://minimaxir.com)

*Last updated: October 17th, 2021*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [1]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

  Preparing metadata (setup.py) ... done


## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first.

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 928Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.41Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 5.53Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:25, 19.9Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 3.23Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.16Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.13Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [5]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [6]:
file_name = "shakespeare.txt"
file_name = "/content/drive/MyDrive/ChatGPT_Paper_wrting/All_Reports/Climate Change _T/Pitch_Deck/Title/Pitch_Deck_T_climate-change-_t.docx" # "path_to_your_file.docx"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies.

In [ ]:


import requests
from bs4 import BeautifulSoup
from gpt_2_simple import gpt_2 as GPT2

def extract_course_information(course_design_variables):
    gpt = GPT2()

    course_data = {}
    course_data['topic'] = gpt.generate(prompt=f"What is the topic of this course? {course_design_variables['topic']}", text=course_design_variables['course_description'])
    course_data['field_of_study'] = gpt.generate(prompt=f"What is the field of study of this course? {course_design_variables['field_of_study']}", text=course_design_variables['course_description'])
    course_data['audience'] = gpt.generate(prompt=f"Who is the target audience for this course? {course_design_variables['audience']}", text=course_design_variables['course_description'])
    course_data['specific_project'] = gpt.generate(prompt=f"What is the specific project or assignment for this course? {course_design_variables['specific_project']}", text=course_design_variables['course_description'])
    course_data['employability'] = gpt.generate(prompt=f"What are the employability or career benefits of this course? {course_design_variables['employability']}", text=course_design_variables['course_description'])
    course_data['practical_skills'] = gpt.generate(prompt=f"What practical skills will students learn in this course? {course_design_variables['practical_skills']}", text=course_design_variables['course_description'])
    course_data['creativity'] = gpt.generate(prompt=f"How does this course encourage creativity? {course_design_variables['creativity']}", text=course_design_variables['course_description'])
    course_data['critical_thinking'] = gpt.generate(prompt=f"How does this course develop critical thinking skills? {course_design_variables['critical_thinking']}", text=course_design_variables['course_description'])
    course_data['fun_and_enjoyment'] = gpt.generate(prompt=f"How does this course make learning engaging and enjoyable? {course_design_variables['fun_and_enjoyment']}", text=course_design_variables['course_description'])
    course_data['employee_guarantee'] = ""
    course_data['collaboration'] = gpt.generate(prompt=f"How does this course foster collaboration among students? {course_design_variables['collaboration']}", text=course_design_variables['course_description'])
    course_data['learning_outcomes'] = gpt.generate(prompt=f"What are the learning outcomes or goals of this course? {course_design_variables['learning_outcomes']}", text=course_design_variables['course_description'])
    course_data['purpose'] = gpt.generate(prompt=f"What is the purpose or objective of this course? {course_design_variables['purpose']}", text=course_design_variables['course_description'])
    course_data['learning_activities'] = gpt.generate(prompt=f"What are the learning activities or methods used in this course? {course_design_variables['learning_activities']}", text=course_design_variables['course_description'])
    course_data['course_content'] = gpt.generate(prompt=f"What are the main topics or subjects covered in this course? {course_design_variables['course_content']}", text=course_design_variables['course_description'])
    course_data['course_assessments'] = gpt.generate(prompt=f"How are students assessed in this course? {course_design_variables['course_assessments']}", text=course_design_variables['course_description'])
    course_data['course_schedule'] = gpt.generate(prompt=f"What is the schedule or timeline for this course? {course_design_variables['course_schedule']}", text=course_design_variables['course_description'])
    course_data['course_sequencing'] = gpt.generate(prompt=f"How is the course content organized or sequenced? {course_design_variables['course_sequencing']}", text=course_design_variables['course_description'])
    course_data['technology_requirements'] = gpt.generate(prompt=f"What technology or equipment is required for this course? {course_design_variables['technology_requirements']}", text=course_design_variables['course_description'])
    course_data['prerequisites'] = gpt.generate(prompt=f"What are the prerequisites or prior knowledge required for this course? {course_design_variables['prerequisites']}", text=course_design_variables['course_description'])

    return course_data

# Example Usage
course_url = "https://uwex.wisconsin.edu/sustainable-management/masters/" # @param {type:"string"} # Wrap the URL in quotes
response = requests.get(course_url)
soup = BeautifulSoup(response.content, 'html.parser')
course_description_element = soup.find('div', class_='course-description__content')

if course_description_element:
    course_description = course_description_element.text.strip()
else:
    course_description = "Description not found on the page"

# Dictionary of course design variables
course_design_variables = course_design_variables

course_design_variables['course_description'] = summary_sumy

course_data = extract_course_information(course_design_variables)
prompts = generate_prompts(course_data)
print(prompts)

In [ ]:
main_variables_0 = {
     'TOPIC': f"{TOPIC}",
     'RESEARCH_DOMAIN': f"{RESEARCH_DOMAIN}",
     'PARAGRAPH': f"{PARAGRAPH}",
     'PARAGRAPHS': f"{PARAGRAPHS}",
     'TOPIC_SENTENCE': f"{TOPIC_SENTENCE}",
     'LANGUAGE': f"{LANGUAGE}",
     'ABSTRACT_PARAGRAPH': f"{ABSTRACT_PARAGRAPH}",
     'BIBLIOGRAPHY': f"{BIBLIOGRAPHY}",
     'THEORY1': f"{THEORY1}",
     'THEORY2': f"{THEORY2}",
     'RESEARCH_QUESTIONS': f"{RESEARCH_QUESTIONS}",
     'ACTION': f"{ACTION}",
     'RESULT_PARAGRAPHS': f"{RESULT_PARAGRAPHS}",
     'DATE': f"{DATE}",
     'NUMBER_OF_DAYS_MONTHS_YEARS': f"{NUMBER_OF_DAYS_MONTHS_YEARS}",
     'role': f"{role}",
     'project_example': f"{project_example}",
     'context': f"{context}",
     'instruction': f"{instruction}",
     'output_format': f"{output_format}",
     'specific_project_details': f"{specific_project_details}",
     'X': f"{X}",
     'project_manager': f"{project_manager}",
     'report': f"{report}",
     'important_themes': f"{important_themes}",
     'project_name': f"{project_name}",
     'stakeholder': f"{stakeholder}",
     'resistant_stakeholder': f"{resistant_stakeholder}",
     'task': f"{task}",
     'Your_Email': f"{Your_Email}"
}

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.54s/it]


dataset has 605362 tokens
Training...
[10 | 845.74] loss=5.92 avg=5.92
[20 | 1658.15] loss=5.60 avg=5.76


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="LORD",
              nsamples=5,
              batch_size=5
              )

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.